In [1]:
#default_exp train3d

In [2]:
#export
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(4)

In [4]:
df_any = Meta.df_any
df_any.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_2a8702d25,CT,ID_ff137633,ID_d17053848c,ID_7098f7c836,,-126.437378,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,1,0,0
ID_66891ac22,CT,ID_42940b2c,ID_17e33f43d0,ID_e14dd0090b,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,0,0,0
ID_8e6e5b51f,CT,ID_76fbed32,ID_1d8eaa14ef,ID_e3919709a0,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,0,0,0,1
ID_cb8b9b514,CT,ID_20039b63,ID_1cfe3e70dd,ID_e3b5d8d9b8,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_ee683911f,CT,ID_a065f3ac,ID_9c727ac231,ID_99e83a310d,,-125.000000,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0


In [5]:
df_slice_count = df_any.groupby(['SeriesInstanceUID']).agg(['count'])
max(df_slice_count.PatientID.values)

array([60])

In [6]:
#export
max_seq_len = 60

In [7]:
#export
class OpenCTs:
    def __init__(self, path): 
        self.fn = get_pil_fn(path)
        self.tt = ToTensor()
    def __call__(self, item):
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        return TensorCTScan(torch.stack(xs))

In [8]:
#export
def pad_batch(x, pad_to=None, value=0):
    bs_pad = pad_to-x.shape[0]
    pad = [0]*len(x.shape)*2
    pad[-1] = bs_pad
    return F.pad(x, pad=pad, value=value)

In [9]:
# pad_batch(x, to=4)

In [10]:
#export
class TfmSOP:
    def __init__(self,df,open_fn,pad_to=None):
        self.open_fn = open_fn
        self.df = df
        self.pad_to = pad_to
    
    def x(self, sid):
        sids = self.df.SOPInstanceUID[sid].values
        x = self.open_fn(sids)
        if self.pad_to is None: return x
        t = type(x)
        return t(pad_batch(x, pad_to=self.pad_to))
    
    def y(self, sid): 
        vals = self.df.loc[sid,htypes].values
        if self.pad_to is not None: 
            vals = pad_batch(tensor(vals), pad_to=self.pad_to, value=-1)
        return TensorMultiCategory(vals).float()

In [11]:
def group_cv(idx, grps): return np.concatenate([grps[o] for o in range_of(grps) if o!=idx])

# column can also be PatientID
def split_data(df, cv_idx, grps, column):
    grp_cv = group_cv(cv_idx, grps)
    mask = df[column].isin(grp_cv)
    return idx[mask],idx[~mask]

In [12]:
#export
def get_3d_dsrc(df, open_fn, grps=Meta.grps, cv_idx=0, column='SeriesInstanceUID',
               pad_to=None):
    df_series = df.reset_index().set_index(column).sort_values("ImagePositionPatient2")
    tfm = TfmSOP(df_series, open_fn, pad_to)
    sids = df_series.index.unique()
    
    s1 = np.where(np.in1d(sids, group_cv(cv_idx,grps)))[0]
    s2 = np.where(np.in1d(sids, grps[cv_idx]))[0]
    dsrc = DataSource(sids, [[tfm.x],[tfm.y]], splits=(s1,s2))
    return dsrc

In [13]:
#export
def get_3d_dbunch(df, path=path_jpg256, bs=None, num_workers=8):
    pad_to = None if bs is None else max_seq_len
    dsrc = get_3d_dsrc(df, open_fn=OpenCTs(path), pad_to=pad_to)

    nrm = Normalize.from_stats(mean,std)
    batch_tfms = L(nrm, Cuda(), IntToFloatTensor())

    dbunch = DataBunch(
        TfmdDL(dsrc.train, bs=bs, after_batch=batch_tfms, num_workers=num_workers, shuffle=True),
        TfmdDL(dsrc.valid, bs=bs, after_batch=batch_tfms, num_workers=num_workers)
    )
    dbunch.device = default_device()
    dbunch.c = 6
    return dbunch


In [14]:
# dsrc = get_3d_dsrc(df_any, open_fn=OpenCTs(path_jpg256), pad_to=max_seq_len)
# x,y = dsrc[0]
# x.shape, y.shape

In [15]:
# dbunch = get_3d_dbunch(df_any, bs=10)
# x,y = dbunch.one_batch()
# x.shape, y.shape

## Features

In [16]:
#export
def get_np_fn(p):
    def _f(fn): return torch.from_numpy(np.load(str(p/f'{fn}.npy')))
    return _f

In [17]:
#export
class OpenFeats:
    def __init__(self, path):
        self.fn = get_np_fn(path)
        self.tt = ToTensor()
    def __call__(self, item):
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        return TensorCTScan(torch.stack(xs))

In [18]:
#export
def get_3d_dbunch_feat(df, path=path/'features_224', bs=1, num_workers=8):
    pad_to = None if bs == 1 else max_seq_len
    dsrc = get_3d_dsrc(df, open_fn=OpenFeats(path), pad_to=pad_to)

    dbunch = DataBunch(
        TfmdDL(dsrc.train, bs=bs, after_batch=[Cuda()], num_workers=num_workers, shuffle=True),
        TfmdDL(dsrc.valid, bs=bs, after_batch=[Cuda()], num_workers=num_workers)
    )
    dbunch.device = default_device()
    dbunch.c = 6
    return dbunch


In [19]:
# dbunch_feat = get_3d_dbunch_feat(df_any, bs=10)
# xb,yb = dbunch_feat.one_batch()
# xb.shape, yb.shape

## Model

In [20]:
#export
class ReshapeCNNBody(Module):
    def __init__(self, body):
        super().__init__()
        self.body = body
        
#     def __setstate__(self, state):
#         self.body.__setstate__(state)
#     def __getstate__(self):
#         return self.body.__getstate__()
    
    def load_state_dict(self, *args, **kwargs):
        self.body.load_state_dict(*args, **kwargs)

    def state_dict(self, *args, **kwargs):
        return self.body.state_dict(*args, **kwargs)
        
    def __getitem__(self, idx):
        return self.body
    
    def forward(self, x):
        shape = x.shape
        x = self.body(x.view(-1, *shape[2:]))
        return x.view(*shape[:2], *x.shape[1:])

In [21]:
class ReshapeBodyHook():
    def __init__(self, body):
        super().__init__()
        body.register_forward_pre_hook(self.pre_hook)
        body.register_forward_hook(self.forward_hook)
        self.shape = None
        
    def pre_hook(self, module, input):
        x = input[0]
        self.shape = x.shape
        return (x.view(-1, *x.shape[2:]),)
    
    def forward_hook(self, module, input, x):
        return x.view(*self.shape[:2], *x.shape[1:])

In [22]:
#export
def conv3(ni,nf,stride=1):
    return ConvLayer(ni, nf, (5,3,3), stride=(1,stride,stride), ndim=3, padding=(2,1,1))

In [23]:
#export
class Batchify(Module):
    def forward(self, x): return x.transpose(1,2)

class DeBatchify(Module):
    def forward(self, x):
        x_t = x.transpose(1,2)
        x_c = x_t.contiguous().view(-1, *x_t.shape[2:])
        return x_c

def get_3d_head():
    m = nn.Sequential(Batchify(),
        conv3(512,256,2), # 8
        conv3(256,128,2), # 4
        conv3(128, 64,2), # 2
        DeBatchify(), nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(64,6))
    init_cnn(m)
    return m

## Ignore Padding in Loss

In [24]:
#export
class DePadLoss(Callback):
    def __init__(self, pad_idx=-1): 
        super().__init__()
        store_attr(self, 'pad_idx')

    def after_pred(self):
        learn = self.learn
        targ = learn.yb[0].view(-1, *learn.yb[0].shape[2:])
        if targ.shape[0] != self.pred.shape[0]:
            pred = learn.pred.view(-1, *learn.pred.shape[2:])
        else: pred = learn.pred
        
        mask = targ[:,-1] != self.pad_idx
        
        learn.pred = pred[mask]
        learn.yb = (targ[mask],)

## Training Features - By Batch

In [24]:
dbunch_feat = get_3d_dbunch_feat(df_any, bs=16)

In [26]:
m = get_3d_head()
learn = get_learner(dbunch_feat, m)

In [27]:
learn.add_cb(DePadLoss())

In [28]:
do_fit(learn, 1, 1e-2)

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.108049,0.104689,0.963100,0.940114,01:22


## Train on Slice

In [25]:
m = get_3d_head()

In [26]:
dbunch = get_3d_dbunch(df_any, bs=10)
config=dict(custom_head=m, init=None)
learn = get_learner(dbunch, resnet18, get_loss(), config=config)

In [27]:
# learn.model[0] = ReshapeCNNBody(learn.model[0])

In [28]:
hook = ReshapeBodyHook(learn.model[0])

In [29]:
learn.add_cb(DePadLoss())

In [ ]:
do_fit(learn, 1, 1e-2)

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time


## Export

In [3]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_data_01_metadata_stage2.ipynb.
Converted 01_data_02_preprocess_windows.ipynb.
Converted 02_train_00_train.ipynb.
Converted 02_train_01_full.ipynb.
Converted 02_train_02_any.ipynb.
Converted 02_train_02c_any_old_double_avg_pool.ipynb.
Converted 03_train3d_00_train3d.ipynb.
Converted 03_train3d_01_train3d-Copy1.ipynb.
Converted 03_train3d_01_train3d-resnet18.ipynb.
Converted 03_train3d_01_train3d.ipynb.
Converted 03_train3d_02_train_head.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 03_train3d_scratch_3Dbatch.ipynb.
Converted 03_train3d_scratch_extract_features.ipynb.
Converted 04_trainSeq_00_trainSeq.ipynb.
